In [7]:
import pandas as pd

# Učitajmo podatke iz CSV fajla
df = pd.read_csv('shoes.csv')

# Opcioni pregled (prvih nekoliko redova)
print(df.head())


   id          name   brand    category  \
0   1   Air Max 270    Nike   Lifestyle   
1   2  Air Jordan 1    Nike  Basketball   
2   3   Ultra Boost  Adidas     Running   
3   4    Stan Smith  Adidas   Lifestyle   
4   5          RS-X    Puma   Lifestyle   

                                         description color  price  
0  Air Max model idealan za svaki dan. Udoban i m...  Crna    120  
1  Čuvena Jordan serija. Retro izgled i vrhunske ...  Bela    150  
2  Maksimalna udobnost prilikom trčanja i svakodn...  Siva    130  
3  Klasični minimalizam uz kožni gornji deo i udo...  Bela     90  
4  Retro-futuristički dizajn, kombinacija više ma...  Crna    100  


In [8]:
def price_category(price):
    if price < 100:
        return "cheap"
    elif price <= 130:
        return "moderate"
    else:
        return "expensive"

# Spajanje feature-a u jedan
def combine_features(row):
    price_cat = price_category(row['price'])
    return f"{row['brand']} {row['category']} {row['description']} {row['color']} {price_cat}"

df['features'] = df.apply(combine_features, axis=1)
print(df['features'])


0    Nike Lifestyle Air Max model idealan za svaki ...
1    Nike Basketball Čuvena Jordan serija. Retro iz...
2    Adidas Running Maksimalna udobnost prilikom tr...
3    Adidas Lifestyle Klasični minimalizam uz kožni...
4    Puma Lifestyle Retro-futuristički dizajn, komb...
5    Puma Lifestyle Legendarni model, prepoznatljiv...
Name: features, dtype: object


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 1. Kreiramo kolonu 'features'
df['features'] = df.apply(combine_features, axis=1)

# 2. Ponovo učimo TF-IDF
tfidf = TfidfVectorizer(
    stop_words=['srpski', 'hrvatski', 'bosanski', 'i', 'u', 'za', 'po', 'od', 'je', 'na', 'iz', 'koji', 'koja', 'koje']
    # ili npr. stop_words='english' ako su vam opisi na engleskom
)
tfidf_matrix = tfidf.fit_transform(df['features'])

# 3. Računamo sličnost
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [10]:
from sklearn.metrics.pairwise import linear_kernel

# Računamo kosinusnu sličnost
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Pogledajmo dimenzije
print(cosine_sim.shape)


(6, 6)


In [11]:
def recommend_shoes(item_id, df, cosine_sim, top_n=5):
    # Pronalaženje indeksa traženog artikla
    item_index = df.index[df['id'] == item_id].tolist()[0]
    
    # Sličnosti za dati artikal
    similarity_scores = list(enumerate(cosine_sim[item_index]))
    
    # Sortiramo po sličnosti
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Uzimamo top_n sličnih artikala (preskačemo prvi, jer je to sam artikal)
    top_similar = similarity_scores[1:top_n+1]
    
    recommended_ids = [df.iloc[i[0]]['id'] for i in top_similar]
    recommended_names = [df.iloc[i[0]]['name'] for i in top_similar]
    
    # Formatiramo povratnu vrednost
    recommendations = list(zip(recommended_ids, recommended_names))
    return recommendations


In [12]:
test_id = 1
recommended = recommend_shoes(test_id, df, cosine_sim, top_n=3)

print(f"Preporuke za artikal sa ID {test_id}:")
for r_id, r_name in recommended:
    print(f"ID: {r_id}, Naziv: {r_name}")


Preporuke za artikal sa ID 1:
ID: 5, Naziv: RS-X
ID: 6, Naziv: Suede Classic
ID: 4, Naziv: Stan Smith
